### 匯入套件

In [271]:
import pandas as pd
import numpy as np
import os


### 匯入檔案

In [272]:
current_dir = os.getcwd()
input_path=os.path.join(current_dir,r"../../data/104_cleaned_0717.csv")

df=pd.read_csv(input_path)
print(df.head())

       上架日期  申請人數        產業            公司名稱  \
0  20250714     1  多媒體傳播相關業  杰德創意影音管理股份有限公司   
1  20250714     6   電腦軟體服務業        不晚科技有限公司   
2  20250715     5      停車場業    台灣國際開發事業有限公司   
3  20250714     3   廣告行銷公關業      創浪行銷顧問有限公司   
4  20250715     5   廣告行銷公關業    華權數位整合行銷有限公司   

                                         description   縣市      職稱      工作編號  \
0  【負責品牌】\r\n全球LGBTQ+娛樂領導平台 GagaOOLala https://ww...  台北市   數據分析師  14540514   
1  1.熟練數據分析工具，能進行數據清理、分析和建模，分析市場營銷活動的效果，提供報告和改進建議...  台北市    數據分析  14325959   
2  1. 串接各式資料來源，進行資料清洗、分析相關商業命題研究。\r\n2. 深入了解業務需求，...  台中市   數據分析師  13103368   
3  數據分析工具監測社群媒體、論壇、新聞平台，定期監控社群輿情。\r\n\r\n2.消費者洞察:...  台北市  數據分析經理  14724608   
4  主要工作述敘：\r\n1. 數據分析與精算：\r\n■分析遊戲玩家行為數據、直播主表現數據，...  台中市   數據分析師  14424533   

   工作型態         經度  ...                學歷要求 是否遠距     薪資上限   薪資下限  工作時段  \
0     2  25.024347  ...           [4, 5, 6]    0    60000  48000   NaN   
1     0  25.076077  ...              [3, 4]    0        0      0   NaN   
2     0  24.160552  

In [273]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363 entries, 0 to 362
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   上架日期         363 non-null    int64  
 1   申請人數         363 non-null    int64  
 2   產業           363 non-null    object 
 3   公司名稱         363 non-null    object 
 4   description  363 non-null    object 
 5   縣市           363 non-null    object 
 6   職稱           363 non-null    object 
 7   工作編號         363 non-null    int64  
 8   工作型態         363 non-null    int64  
 9   經度           363 non-null    float64
 10  緯度           363 non-null    float64
 11  major        363 non-null    object 
 12  學歷要求         363 non-null    object 
 13  是否遠距         363 non-null    int64  
 14  薪資上限         363 non-null    int64  
 15  薪資下限         363 non-null    int64  
 16  工作時段         137 non-null    object 
 17  職務分類代碼       363 non-null    object 
 18  是否積極徵才       363 non-null    int64  
 19  條件要求    

In [274]:
df.isnull().sum()

上架日期             0
申請人數             0
產業               0
公司名稱             0
description      0
縣市               0
職稱               0
工作編號             0
工作型態             0
經度               0
緯度               0
major            0
學歷要求             0
是否遠距             0
薪資上限             0
薪資下限             0
工作時段           226
職務分類代碼           0
是否積極徵才           0
條件要求             0
福利制度             0
員工人數             0
dtype: int64

### 工作時段太多Null值，刪除該欄位

In [275]:
df.drop(columns=['工作時段','工作型態','職務分類代碼','薪資上限','福利制度'],inplace=True)

In [276]:
df['薪資是否為面議']= np.where(df['薪資下限']==0,1,0) #增加'薪資是否為面議'欄位
df['薪資下限']=df['薪資下限'].replace(0,40000)    #將面議的改成法規規定最低40000
df=df[df['薪資下限']>=1000]     #將時薪(非正職、學生實習)的資料去除

#自動判斷月薪超過300000為年薪，並除以12，轉為月薪
df['薪資下限'] = df['薪資下限'].astype(float)   #原本欄位是(int)類型，除12後會變float，會相衝，故先轉成float。
df.loc[df['薪資下限'] > 300000, '薪資下限']/= 12
df['薪資下限'] = df['薪資下限'].round(0).astype(int)


In [277]:
df.shape

(350, 18)

In [278]:
df['major']

0                                        []
1                                        []
2                                        []
3            ['統計學相關', '資訊管理相關', '大眾傳播學相關']
4             ['統計學相關', '數理統計相關', '應用數學相關']
                       ...                 
358          ['資訊工程相關', '資訊管理相關', '數理統計相關']
359    ['資訊工程相關', '數理統計相關', '其他數學及電算機科學相關']
360            ['資訊管理相關', '統計學相關', '經濟學相關']
361                                      []
362            ['數理統計相關', '統計學相關', '經濟學相關']
Name: major, Length: 350, dtype: object

In [279]:
import ast # 導入 ast 模組
df["是否要求特定major"] = df['major'].apply(lambda x: 1 if len(ast.literal_eval(x)) > 0 else 0) #增加一欄，[是否要求特定major]的欄位。

In [280]:
df['是否要求特定major'].tail(20)

343    1
344    0
345    0
346    0
347    0
348    0
349    0
350    0
351    1
352    1
353    1
354    0
355    1
356    0
357    1
358    1
359    1
360    1
361    0
362    1
Name: 是否要求特定major, dtype: int64

In [281]:
df['是否遠距'].value_counts()

是否遠距
0    323
2     24
1      3
Name: count, dtype: int64

In [282]:
df['是否遠距'] = df['是否遠距'].astype(int)

In [283]:
remote_work={0:"無遠距",1:"完全遠距",2:"混和"}
df['是否遠距']=df['是否遠距'].map(remote_work)

In [284]:
df.head()

,上架日期,申請人數,產業,公司名稱,description,縣市,職稱,工作編號,經度,緯度,major,學歷要求,是否遠距,薪資下限,是否積極徵才,條件要求,員工人數,薪資是否為面議,是否要求特定major
0,20250714,1,多媒體傳播相關業,杰德創意影音管理股份有限公司,【負責品牌】\r\n全球LGBTQ+娛樂領導平台 GagaOOLala https://ww...,台北市,數據分析師,14540514,25.024347,121.548339,[],"[4, 5, 6]",無遠距,48000,1,"{'acceptRole': {'role': [{'code': 32, 'descrip...",暫不提供,0,0
1,20250714,6,電腦軟體服務業,不晚科技有限公司,1.熟練數據分析工具，能進行數據清理、分析和建模，分析市場營銷活動的效果，提供報告和改進建議...,台北市,數據分析,14325959,25.076077,121.575436,[],"[3, 4]",無遠距,40000,1,"{'acceptRole': {'role': [], 'disRole': {'needH...",20人,1,0
2,20250715,5,停車場業,台灣國際開發事業有限公司,1. 串接各式資料來源，進行資料清洗、分析相關商業命題研究。\r\n2. 深入了解業務需求，...,台中市,數據分析師,13103368,24.160552,120.648480,[],"[1, 2, 3, 4, 5, 6]",無遠距,40000,1,"{'acceptRole': {'role': [{'code': 2, 'descript...",100人,0,0
3,20250714,3,廣告行銷公關業,創浪行銷顧問有限公司,數據分析工具監測社群媒體、論壇、新聞平台，定期監控社群輿情。\r\n\r\n2.消費者洞察:...,台北市,數據分析經理,14724608,25.034500,121.560053,"['統計學相關', '資訊管理相關', '大眾傳播學相關']","[4, 5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",暫不提供,1,1
4,20250715,5,廣告行銷公關業,華權數位整合行銷有限公司,主要工作述敘：\r\n1. 數據分析與精算：\r\n■分析遊戲玩家行為數據、直播主表現數據，...,台中市,數據分析師,14424533,24.116516,120.615540,"['統計學相關', '數理統計相關', '應用數學相關']","[4, 5, 6]",無遠距,40000,1,"{'acceptRole': {'role': [{'code': 2, 'descript...",暫不提供,0,1


In [285]:
df[df["工作編號"].duplicated()]

,上架日期,申請人數,產業,公司名稱,description,縣市,職稱,工作編號,經度,緯度,major,學歷要求,是否遠距,薪資下限,是否積極徵才,條件要求,員工人數,薪資是否為面議,是否要求特定major


In [286]:
df[df["工作編號"].isin([14720195,13634345])]

,上架日期,申請人數,產業,公司名稱,description,縣市,職稱,工作編號,經度,緯度,major,學歷要求,是否遠距,薪資下限,是否積極徵才,條件要求,員工人數,薪資是否為面議,是否要求特定major
177,20250716,23,家用電器製造業,金筆企業股份有限公司,1. 工廠生產數據收集、整理與分析，協助相關人員了解生產狀況\r\n2. 建立分析指標，追蹤...,新北市,數據分析工程師,14720195,25.068090,121.455088,"['數理統計相關', '其他數學及電算機科學相關', '工業工程相關']",[4],無遠距,38000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",108人,0,1
359,20250715,20,光電產業,達興材料股份有限公司,1. 執行AI技術開發專案。\r\n2. 大數據分析與建模。\r\n3. 舉辦內部統計與數值...,台中市,研發-資料科學研究員,13634345,24.204932,120.607608,"['資訊工程相關', '數理統計相關', '其他數學及電算機科學相關']","[5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [{'code': 2, 'descript...",430人,1,1


In [287]:
df.drop_duplicates(subset="工作編號",keep="first",inplace=True)  #保留第一個

In [288]:
df.set_index("工作編號")

,上架日期,申請人數,產業,公司名稱,description,縣市,職稱,經度,緯度,major,學歷要求,是否遠距,薪資下限,是否積極徵才,條件要求,員工人數,薪資是否為面議,是否要求特定major
工作編號,,,,,,,,,,,,,,,,,,
14540514,20250714,1,多媒體傳播相關業,杰德創意影音管理股份有限公司,【負責品牌】\r\n全球LGBTQ+娛樂領導平台 GagaOOLala https://ww...,台北市,數據分析師,25.024347,121.548339,[],"[4, 5, 6]",無遠距,48000,1,"{'acceptRole': {'role': [{'code': 32, 'descrip...",暫不提供,0,0
14325959,20250714,6,電腦軟體服務業,不晚科技有限公司,1.熟練數據分析工具，能進行數據清理、分析和建模，分析市場營銷活動的效果，提供報告和改進建議...,台北市,數據分析,25.076077,121.575436,[],"[3, 4]",無遠距,40000,1,"{'acceptRole': {'role': [], 'disRole': {'needH...",20人,1,0
13103368,20250715,5,停車場業,台灣國際開發事業有限公司,1. 串接各式資料來源，進行資料清洗、分析相關商業命題研究。\r\n2. 深入了解業務需求，...,台中市,數據分析師,24.160552,120.648480,[],"[1, 2, 3, 4, 5, 6]",無遠距,40000,1,"{'acceptRole': {'role': [{'code': 2, 'descript...",100人,0,0
14724608,20250714,3,廣告行銷公關業,創浪行銷顧問有限公司,數據分析工具監測社群媒體、論壇、新聞平台，定期監控社群輿情。\r\n\r\n2.消費者洞察:...,台北市,數據分析經理,25.034500,121.560053,"['統計學相關', '資訊管理相關', '大眾傳播學相關']","[4, 5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",暫不提供,1,1
14424533,20250715,5,廣告行銷公關業,華權數位整合行銷有限公司,主要工作述敘：\r\n1. 數據分析與精算：\r\n■分析遊戲玩家行為數據、直播主表現數據，...,台中市,數據分析師,24.116516,120.615540,"['統計學相關', '數理統計相關', '應用數學相關']","[4, 5, 6]",無遠距,40000,1,"{'acceptRole': {'role': [{'code': 2, 'descript...",暫不提供,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375020,20250623,17,IC設計相關業,華邦電子股份有限公司,\r\n立即填寫完整履歷投遞華邦電子2025校園徵才職缺，將有機會讓你Win在起跑點、提早卡...,台中市,華邦電子2025校園徵才-(竹北/中科)軟體類_大數據分析及演算法開發工程師,24.212104,120.619947,"['資訊工程相關', '資訊管理相關', '數理統計相關']","[4, 5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [{'code': 2, 'descript...",2800人,1,1
13634345,20250715,20,光電產業,達興材料股份有限公司,1. 執行AI技術開發專案。\r\n2. 大數據分析與建模。\r\n3. 舉辦內部統計與數值...,台中市,研發-資料科學研究員,24.204932,120.607608,"['資訊工程相關', '數理統計相關', '其他數學及電算機科學相關']","[5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [{'code': 2, 'descript...",430人,1,1
14516906,20250714,13,電腦系統整合服務業,陣列科技有限公司,主責參與資料分析專案、建置儀錶板，協助使用者從資料中獲取洞察結果，落實數據驅動決策，持續優化...,台北市,外商（FMCG）資料分析人員,25.040920,121.572006,"['資訊管理相關', '統計學相關', '經濟學相關']","[3, 4, 5]",無遠距,40000,1,"{'acceptRole': {'role': [{'code': 2, 'descript...",20人,1,1


In [289]:
from cleaning_industry import clean_industry_column
df=clean_industry_column(df)
df.drop(columns=["產業"],inplace=True)

In [290]:
df["八大產業"].value_counts()

八大產業
資訊科技業       99
服務與專業顧問業    76
金融與保險業      65
製造業         48
批發與零售業      34
醫療與健康照護     14
觀光與餐飲業       7
運輸與物流業       5
其他           2
Name: count, dtype: int64

In [291]:
df.head()

,上架日期,申請人數,公司名稱,description,縣市,職稱,工作編號,經度,緯度,major,學歷要求,是否遠距,薪資下限,是否積極徵才,條件要求,員工人數,薪資是否為面議,是否要求特定major,八大產業
0,20250714,1,杰德創意影音管理股份有限公司,【負責品牌】\r\n全球LGBTQ+娛樂領導平台 GagaOOLala https://ww...,台北市,數據分析師,14540514,25.024347,121.548339,[],"[4, 5, 6]",無遠距,48000,1,"{'acceptRole': {'role': [{'code': 32, 'descrip...",暫不提供,0,0,服務與專業顧問業
1,20250714,6,不晚科技有限公司,1.熟練數據分析工具，能進行數據清理、分析和建模，分析市場營銷活動的效果，提供報告和改進建議...,台北市,數據分析,14325959,25.076077,121.575436,[],"[3, 4]",無遠距,40000,1,"{'acceptRole': {'role': [], 'disRole': {'needH...",20人,1,0,資訊科技業
2,20250715,5,台灣國際開發事業有限公司,1. 串接各式資料來源，進行資料清洗、分析相關商業命題研究。\r\n2. 深入了解業務需求，...,台中市,數據分析師,13103368,24.160552,120.648480,[],"[1, 2, 3, 4, 5, 6]",無遠距,40000,1,"{'acceptRole': {'role': [{'code': 2, 'descript...",100人,0,0,運輸與物流業
3,20250714,3,創浪行銷顧問有限公司,數據分析工具監測社群媒體、論壇、新聞平台，定期監控社群輿情。\r\n\r\n2.消費者洞察:...,台北市,數據分析經理,14724608,25.034500,121.560053,"['統計學相關', '資訊管理相關', '大眾傳播學相關']","[4, 5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",暫不提供,1,1,服務與專業顧問業
4,20250715,5,華權數位整合行銷有限公司,主要工作述敘：\r\n1. 數據分析與精算：\r\n■分析遊戲玩家行為數據、直播主表現數據，...,台中市,數據分析師,14424533,24.116516,120.615540,"['統計學相關', '數理統計相關', '應用數學相關']","[4, 5, 6]",無遠距,40000,1,"{'acceptRole': {'role': [{'code': 2, 'descript...",暫不提供,0,1,服務與專業顧問業


In [292]:
df

,上架日期,申請人數,公司名稱,description,縣市,職稱,工作編號,經度,緯度,major,學歷要求,是否遠距,薪資下限,是否積極徵才,條件要求,員工人數,薪資是否為面議,是否要求特定major,八大產業
0,20250714,1,杰德創意影音管理股份有限公司,【負責品牌】\r\n全球LGBTQ+娛樂領導平台 GagaOOLala https://ww...,台北市,數據分析師,14540514,25.024347,121.548339,[],"[4, 5, 6]",無遠距,48000,1,"{'acceptRole': {'role': [{'code': 32, 'descrip...",暫不提供,0,0,服務與專業顧問業
1,20250714,6,不晚科技有限公司,1.熟練數據分析工具，能進行數據清理、分析和建模，分析市場營銷活動的效果，提供報告和改進建議...,台北市,數據分析,14325959,25.076077,121.575436,[],"[3, 4]",無遠距,40000,1,"{'acceptRole': {'role': [], 'disRole': {'needH...",20人,1,0,資訊科技業
2,20250715,5,台灣國際開發事業有限公司,1. 串接各式資料來源，進行資料清洗、分析相關商業命題研究。\r\n2. 深入了解業務需求，...,台中市,數據分析師,13103368,24.160552,120.648480,[],"[1, 2, 3, 4, 5, 6]",無遠距,40000,1,"{'acceptRole': {'role': [{'code': 2, 'descript...",100人,0,0,運輸與物流業
3,20250714,3,創浪行銷顧問有限公司,數據分析工具監測社群媒體、論壇、新聞平台，定期監控社群輿情。\r\n\r\n2.消費者洞察:...,台北市,數據分析經理,14724608,25.034500,121.560053,"['統計學相關', '資訊管理相關', '大眾傳播學相關']","[4, 5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [], 'disRole': {'needH...",暫不提供,1,1,服務與專業顧問業
4,20250715,5,華權數位整合行銷有限公司,主要工作述敘：\r\n1. 數據分析與精算：\r\n■分析遊戲玩家行為數據、直播主表現數據，...,台中市,數據分析師,14424533,24.116516,120.615540,"['統計學相關', '數理統計相關', '應用數學相關']","[4, 5, 6]",無遠距,40000,1,"{'acceptRole': {'role': [{'code': 2, 'descript...",暫不提供,0,1,服務與專業顧問業
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,20250623,17,華邦電子股份有限公司,\r\n立即填寫完整履歷投遞華邦電子2025校園徵才職缺，將有機會讓你Win在起跑點、提早卡...,台中市,華邦電子2025校園徵才-(竹北/中科)軟體類_大數據分析及演算法開發工程師,14375020,24.212104,120.619947,"['資訊工程相關', '資訊管理相關', '數理統計相關']","[4, 5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [{'code': 2, 'descript...",2800人,1,1,資訊科技業
359,20250715,20,達興材料股份有限公司,1. 執行AI技術開發專案。\r\n2. 大數據分析與建模。\r\n3. 舉辦內部統計與數值...,台中市,研發-資料科學研究員,13634345,24.204932,120.607608,"['資訊工程相關', '數理統計相關', '其他數學及電算機科學相關']","[5, 6]",無遠距,40000,0,"{'acceptRole': {'role': [{'code': 2, 'descript...",430人,1,1,製造業
360,20250714,13,陣列科技有限公司,主責參與資料分析專案、建置儀錶板，協助使用者從資料中獲取洞察結果，落實數據驅動決策，持續優化...,台北市,外商（FMCG）資料分析人員,14516906,25.040920,121.572006,"['資訊管理相關', '統計學相關', '經濟學相關']","[3, 4, 5]",無遠距,40000,1,"{'acceptRole': {'role': [{'code': 2, 'descript...",20人,1,1,資訊科技業
361,20250714,13,Advantech_研華股份有限公司,[職務概述]\r\n\r\n研華的數據中台已具一定規模，正邁向下一階段的效能優化與架構升級。...,台北市,BI/Data Platform Senior Engineer - 內湖_JR202505011,14628991,25.069263,121.582397,[],"[4, 5, 6]",無遠距,40000,1,"{'acceptRole': {'role': [], 'disRole': {'needH...",9430人,1,0,資訊科技業


In [293]:
df["學歷要求"].value_counts()

學歷要求
[4, 5, 6]             118
[4, 5]                102
[3, 4, 5, 6]           32
[1, 2, 3, 4, 5, 6]     25
[3, 4, 5]              20
[4]                    16
[5]                    13
[5, 6]                  9
[2, 3, 4, 5, 6]         7
[3, 4]                  5
[2, 3, 4, 5]            1
[2, 3, 4]               1
[6]                     1
Name: count, dtype: int64

In [294]:
edu_map = {
            1: '不拘',
            2: '高中',
            3: '專科',
            4: '大學',
            5: '碩士',
            6: '博士',
        }
df['學歷要求']=df['學歷要求'].apply(lambda x:edu_map.get(min(ast.literal_eval(x))))

In [295]:
df['員工人數'] = df['員工人數'].apply(lambda x: int(x.replace('人', '')) if isinstance(x, str) and x.endswith('人') else pd.NA) # 去'人'轉數字，並將'暫不提供'改為NA

In [296]:
#增加[公司規模]欄位，依員工人數劃分
def classify_company_size(num):       
    if pd.isnull(num):
        return '未知'
    elif num<50:
        return '小型'
    elif num < 200:
        return '中型'
    else:
        return '大型'
df['公司規模']=df['員工人數'].apply(classify_company_size)

In [297]:
df.tail()

,上架日期,申請人數,公司名稱,description,縣市,職稱,工作編號,經度,緯度,major,學歷要求,是否遠距,薪資下限,是否積極徵才,條件要求,員工人數,薪資是否為面議,是否要求特定major,八大產業,公司規模
358,20250623,17,華邦電子股份有限公司,\r\n立即填寫完整履歷投遞華邦電子2025校園徵才職缺，將有機會讓你Win在起跑點、提早卡...,台中市,華邦電子2025校園徵才-(竹北/中科)軟體類_大數據分析及演算法開發工程師,14375020,24.212104,120.619947,"['資訊工程相關', '資訊管理相關', '數理統計相關']",大學,無遠距,40000,0,"{'acceptRole': {'role': [{'code': 2, 'descript...",2800,1,1,資訊科技業,大型
359,20250715,20,達興材料股份有限公司,1. 執行AI技術開發專案。\r\n2. 大數據分析與建模。\r\n3. 舉辦內部統計與數值...,台中市,研發-資料科學研究員,13634345,24.204932,120.607608,"['資訊工程相關', '數理統計相關', '其他數學及電算機科學相關']",碩士,無遠距,40000,0,"{'acceptRole': {'role': [{'code': 2, 'descript...",430,1,1,製造業,大型
360,20250714,13,陣列科技有限公司,主責參與資料分析專案、建置儀錶板，協助使用者從資料中獲取洞察結果，落實數據驅動決策，持續優化...,台北市,外商（FMCG）資料分析人員,14516906,25.040920,121.572006,"['資訊管理相關', '統計學相關', '經濟學相關']",專科,無遠距,40000,1,"{'acceptRole': {'role': [{'code': 2, 'descript...",20,1,1,資訊科技業,小型
361,20250714,13,Advantech_研華股份有限公司,[職務概述]\r\n\r\n研華的數據中台已具一定規模，正邁向下一階段的效能優化與架構升級。...,台北市,BI/Data Platform Senior Engineer - 內湖_JR202505011,14628991,25.069263,121.582397,[],大學,無遠距,40000,1,"{'acceptRole': {'role': [], 'disRole': {'needH...",9430,1,0,資訊科技業,大型
362,20250717,26,Human8_香港商因賽思股份有限公司台灣分公司,[Purpose of Role]\r\n\r\nThe Data Processor is...,台北市,Data Analyst / Data Processer,14722723,25.036180,121.569282,"['數理統計相關', '統計學相關', '經濟學相關']",大學,無遠距,40000,1,"{'acceptRole': {'role': [], 'disRole': {'needH...",20,1,1,服務與專業顧問業,小型


In [298]:
df['是否遠距'].value_counts()

是否遠距
無遠距     323
混和       24
完全遠距      3
Name: count, dtype: int64

In [299]:
#暫時匯出
#desktop_path = os.path.join(os.path.expanduser("~"), "Desktop", "暫時匯出查看.csv")
#df.to_csv(desktop_path, index=False, encoding='utf-8-sig')